**<font size = 6>**
# This is the `Demo.ipynb` file

I will demonstrate how to use my classes and functions.

In [1]:
import random
import torch
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

**<font size = 4>**
# The `UCIDatasets` loader in `/utils/dataframe.py `

`reference: https://gist.github.com/martinferianc/db7615c85d5a3a71242b4916ea6a14a2`

Note that the output `train` or `test` is pytorch.Dataset

In [2]:
from utils.dataframe import UCIDatasets, datalist
print(datalist)
data = UCIDatasets("housing")
train = data.get_split( load="train") #pytorch.dataset
test = data.get_split( load="test")

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
train_loader = data.get_split_dataloader(load = "train", batch_size = 16) #pytorch.dataloader

['housing', 'concrete', 'energy', 'power', 'redwine', 'whitewine', 'yacht']


**<font size = 4>**
# The `imputer` in `model/imputer.py`.

I assemble several common methods, some detail can see my code and the documentary of sklearn.impute. 

I have set up some parameters. change the parameters via `par_setting( par_dict )` function.

The input dictionary should be like `__getParvalue__()`

In [3]:
from model.imputer import imputer, method_list
import numpy as np
print(method_list)
X = np.random.rand(100*20).reshape(100,20)
X[51:, 11:] = np.nan

impObject = imputer(X, method = 'mice')
impObject.train()
imp = impObject.imp
imp.transform(X).shape 
print('\n\nSet up parameters via impObject.par_setting( par_dict ) and retrain the model, \
    the par_dict should be like the following structure:\n {} \n'.format(impObject.get_Parvalue()))
print('for instance: impObject.par_setting( { \'max_iter\': 10 } ) ')
print('\nEach par corresponding to method is as the following:')
impObject.get_Parlist()
print('where SimpleImputer includes [\'mean\', \'median\', \'most_frequent\']')


['mean', 'median', 'most_frequent', 'mice', 'missForest', 'knn']


Set up parameters via impObject.par_setting( par_dict ) and retrain the model,     the par_dict should be like the following structure:
 {'missing_values': nan, 'max_iter': 10, 'random_state': 0, 'n_estimators': 100, 'n_neighbors': 3, 'metric': 'nan_euclidean'} 

for instance: impObject.par_setting( { 'max_iter': 10 } ) 

Each par corresponding to method is as the following:
where SimpleImputer includes ['mean', 'median', 'most_frequent']


**<font size = 4>**
# This part is for `MIWAE` in `/model/MIWAE.py` and `trainer` in `/utils/trainer.py`.

`http://proceedings.mlr.press/v97/mattei19a/mattei19a.pdf (ICML, 2019)`.

`MIWAE` is a pytorch model

Might use `trainer` to train it.

Loss function are like `loss(self, outdic, indic)` in which outdic and indic are input and output

For `MIWAE`, 

`indic = {'x': x , 'm': m }` where `x` is dataset and `m` is missing indicator.

`output = {'lpxz': lpxz , 'lqzx': lqzx , 'lpz': lpz }`  means `l`og loss for `p`(`x` | `z`), `q`(`z` | `x`), `p`(`z`)

In MIWAE loss `self.MIWAE_ELBO(outdic, indic = None)` the indic is not required.


In [4]:
from model.MIWAE import MIWAE
from utils.trainer import VAE_trainer
from utils.dataframe import UCIDatasets, datalist
data = UCIDatasets("whitewine")

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
train_loader = data.get_split_dataloader(load = "train", batch_size = 16) #pytorch.dataloader
test_loader = data.get_split_dataloader(load = "test", batch_size = 16) #pytorch.dataloader
model = MIWAE(data_dim = 20, n_samples=5, permutation_invariance=True)
trainer = VAE_trainer(model = model, train_loader = train_loader, test_loader = test_loader)
trainer.model_summary()

Es torch.Size([16, 20, 20])
Esx torch.Size([16, 20, 21])
Esxr torch.Size([320, 21])
h torch.Size([320, 20])
hr torch.Size([16, 20, 20])
hz torch.Size([16, 20, 20])
g torch.Size([16, 20])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 20]             440
            Linear-2                  [-1, 100]           2,100
              Tanh-3                  [-1, 100]               0
            Linear-4                  [-1, 100]          10,100
              Tanh-5                  [-1, 100]               0
            Linear-6                   [-1, 50]           5,050
            Linear-7                   [-1, 50]           5,050
            Linear-8               [-1, 5, 100]           5,100
            Linear-9               [-1, 5, 100]          10,100
           Linear-10                [-1, 5, 20]           2,020
           Linear-11                [-1, 5, 

**<font size = 4>**
# A simple example

Here is a simple example using `VAE_trainer` class from `/utils/trainer.py` to train `VAE` on `MNIST`.

First, we may set some hyperparameters.

Then read the training data and validation data (`train_loader` structure) and put them into trainer.

The trainer will use `VAE_loss` automatically for `VAE` model.

I will use this trainer for all `VAE` like models in this project.

In [5]:
import os
import torch
import torch.utils.data
import random
import pathlib
import gc
import numpy as np
from torchvision import datasets, transforms
from model.VAE import VAE
from utils.trainer import VAE_trainer
from utils.experiment import fs_setup, check_training_file

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
######################################
# settings
batch_size=128
max_epochs=52
no_cuda = False
seed=1
log_interval=10
cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if cuda else "cpu")
"""
Setup directory of Demo
"""
config = {'batch_size': batch_size }
experiment_dir = fs_setup('Demo', seed, config)
expr_file = experiment_dir / f'VAE.npz'
check_point = experiment_dir / f'VAE_ckpt.pth'
###############
# Read history loss file if exist
###############
train_file = check_training_file(expr_file)
history, start_epoch = train_file['history'], train_file['start_epoch']
if start_epoch >= max_epochs:
    print('skipping {} (seed={})   start_epoch({}), num_of_epoch({})'.format('Demo', seed, start_epoch, max_epochs))
del train_file
gc.collect()
###############
# set kwargs for loading checkpoint if exist
###############
train_kwargs = {
    'check_point': check_point, 'expr_file': expr_file, 'start_epoch': start_epoch, 'history': history,
    } 
# end settings
######################################
"""
Make data loader
"""
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, num_workers = 1, pin_memory = True )

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, num_workers = 1, pin_memory = True)
"""
Train model
"""
model = VAE()
trainer = VAE_trainer(model = model, train_loader = train_loader, test_loader= test_loader, 
            **config, **train_kwargs,)
trainer.train(max_epochs = max_epochs)

Train Epoch: 51 [0/60000 (0%)]	Loss: 99.689117
Train Epoch: 51 [1280/60000 (2%)]	Loss: 103.431137
Train Epoch: 51 [2560/60000 (4%)]	Loss: 104.766655
Train Epoch: 51 [3840/60000 (6%)]	Loss: 99.233307
Train Epoch: 51 [5120/60000 (9%)]	Loss: 101.182518
Train Epoch: 51 [6400/60000 (11%)]	Loss: 102.850739
Train Epoch: 51 [7680/60000 (13%)]	Loss: 101.406624
Train Epoch: 51 [8960/60000 (15%)]	Loss: 100.912262
Train Epoch: 51 [10240/60000 (17%)]	Loss: 102.272636
Train Epoch: 51 [11520/60000 (19%)]	Loss: 103.237030
Train Epoch: 51 [12800/60000 (21%)]	Loss: 99.779877
Train Epoch: 51 [14080/60000 (23%)]	Loss: 100.001404
Train Epoch: 51 [15360/60000 (26%)]	Loss: 107.487274
Train Epoch: 51 [16640/60000 (28%)]	Loss: 100.886490
Train Epoch: 51 [17920/60000 (30%)]	Loss: 103.306786
Train Epoch: 51 [19200/60000 (32%)]	Loss: 100.287735
Train Epoch: 51 [20480/60000 (34%)]	Loss: 103.765167
Train Epoch: 51 [21760/60000 (36%)]	Loss: 98.166534
Train Epoch: 51 [23040/60000 (38%)]	Loss: 101.741867
Train Epoch: 

**<font size = 4>**
# Reproduce the experiment in the paper notMIWAE

I will follow the setting in the paper https://arxiv.org/pdf/2006.12871.pdf

The experiment code is in `exp_imputation` (see `utils/experiment.py`)

Use `UCIDatasets` to process the data

Use `imputer` to train the `mean`, `MissForest`, `MICE`

Use `VAE_trainer` to train the `miwae`, `notmiwae`

Store check_points every 50 epochs, including `model.state_dict()` and `optimize.state_dict()`.

In [6]:
"""
Use the MIWAE and not-MIWAE on UCI data
"""
import random
import torch
import numpy as np
import os
#import sys
from utils.dataframe import UCIDatasets
from utils.experiment import exp_imputation
#sys.path.append(os.getcwd())
"""
Follow the amputation setting and data settings in https://github.com/nbip/notMIWAE/blob/master/task01.py
"""
# ---- data settings
name = 'whitewine'
n_hidden = 128
n_samples = 20
max_iter = 100000
batch_size = 16
impute_sample = 10000

###   the missing model   ###
# mprocess = 'linear'
# mprocess = 'selfmasking'
mprocess = 'selfmasking_known'

# ---- number of runs
runs = 1
RMSE_result = dict()
methods = ['miwae','notmiwae','mean','mice','RF']
for method in methods:
    RMSE_result[method] = []
"""
load data: white wine
"""
data = UCIDatasets(name=name)
N, D = data.N, data.D
dl = D - 1
optim_kwargs = {'lr': 0.0001, 'betas': (0.9, 0.999), 'eps': 1e-08 }
MIWAE_kwargs = {
    'data_dim': D, 'z_dim': dl, 'h_dim': n_hidden, 'n_samples': n_samples
    }
notMIWAE_kwargs = {
    'data_dim': D, 'z_dim': dl, 'h_dim': n_hidden, 'n_samples': n_samples, 'missing_process': mprocess
    }
data_kwargs = {
    'batch_size': batch_size
    }
imputer_par = {
    'missing_values': np.nan, 'max_iter': 10, 'random_state': 0, 'n_estimators': 100, 'n_neighbors': 3, 'metric': 'nan_euclidean'
    }
exp_kwargs = {
    'dataset':name, 'runs':runs, 'seed': seed, 'impute_sample': impute_sample,
}
config = {
    'exp_kwargs': exp_kwargs, 'optim_kwargs': optim_kwargs,
    'MIWAE_kwargs': MIWAE_kwargs, 'notMIWAE_kwargs': notMIWAE_kwargs,
    'data_kwargs': data_kwargs, 'imputer_par': imputer_par,
    }
RMSE_result = exp_imputation( 'exp_imputation', model_list = ['miwae', 'notmiwae'], config = config, num_of_epoch = max_iter)

print("RMSE_miwae = {0:.5f} +- {1:.5f}".format(np.mean(RMSE_result['miwae']), np.std(RMSE_result['miwae'])))
print("RMSE_notmiwae = {0:.5f} +- {1:.5f}".format(np.mean(RMSE_result['notmiwae']), np.std(RMSE_result['notmiwae'])))
print("RMSE_mean = {0:.5f} +- {1:.5f}".format(np.mean(RMSE_result['mean']), np.std(RMSE_result['mean'])))
print("RMSE_mice = {0:.5f} +- {1:.5f}".format(np.mean(RMSE_result['mice']), np.std(RMSE_result['mice'])))
print("RMSE_missForest = {0:.5f} +- {1:.5f}".format(np.mean(RMSE_result['RF']), np.std(RMSE_result['RF'])))

Train Epoch: 10151 [0/4897 (0%)]	Loss: -18.102787
Train Epoch: 10151 [160/4897 (3%)]	Loss: -16.826050
Train Epoch: 10151 [320/4897 (7%)]	Loss: -22.099731
Train Epoch: 10151 [480/4897 (10%)]	Loss: -19.928675
Train Epoch: 10151 [640/4897 (13%)]	Loss: -20.981087
Train Epoch: 10151 [800/4897 (16%)]	Loss: -21.011297
Train Epoch: 10151 [960/4897 (20%)]	Loss: -22.922094
Train Epoch: 10151 [1120/4897 (23%)]	Loss: -22.524921
Train Epoch: 10151 [1280/4897 (26%)]	Loss: -23.510267
Train Epoch: 10151 [1440/4897 (29%)]	Loss: -17.749022
Train Epoch: 10151 [1600/4897 (33%)]	Loss: -16.829840
Train Epoch: 10151 [1760/4897 (36%)]	Loss: -17.391521
Train Epoch: 10151 [1920/4897 (39%)]	Loss: -21.467682
Train Epoch: 10151 [2080/4897 (42%)]	Loss: -25.092234
Train Epoch: 10151 [2240/4897 (46%)]	Loss: -21.024401
Train Epoch: 10151 [2400/4897 (49%)]	Loss: -14.670938
Train Epoch: 10151 [2560/4897 (52%)]	Loss: -26.113281
Train Epoch: 10151 [2720/4897 (55%)]	Loss: -22.954977
Train Epoch: 10151 [2880/4897 (59%)]	Los

ValueError: Expected parameter loc (Tensor of shape (16, 10)) of distribution Normal(loc: torch.Size([16, 10]), scale: torch.Size([16, 10])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, 

**<font size = 4>**
# Some issues here

I follow all settings from the paper, a big difference is I use pytorch and the original paper use tensorflow 1

The encoder seems has some problem (need to fix it)

The original paper use clip as activation function for std in the encoder.

Perhaps torch.clamp has a little difference I haven't noticed.